# Beyond Single Feature Importance with ICECREAM: Credit Experiment

In this experiment (see also Section 5.1 of the paper), we apply ICECREAM to the South German Credit dataset by Grömping (2019).

The notebook is meant to enable the user to re-produce our results, either by fully repeating all calculations, or by loading the results of the time-consuming tasks from the `/data` folder and only creating the graphs. Therefore, the notebook is split into four sections:
- Setup: This section imports the required modules and initializes the `shap` package for visual output.

- Preparation: This section prepares the dataset (since `shap` cannot work with string-formatted categorical features), trains the base model and computes the SHAP values. It also stores all generated data in a folder (the default is `/data`), and can therefore be skipped when executing the notebook.

- Calculation of explanation scores: This is the calculation of minimal full-explanation coalitions with ICECREAM. The `/original_data` folder already contains the original results of this calculation, so it can also be skipped.

- Analysis: This section loads the data from the previous sections and performs the same analysis we are showing in the paper.

## Setup

In [ ]:
import random
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn import set_config
import shap

from explain.explanations.model_explanation import find_minimum_size_coalitions

In [ ]:
shap.initjs()
set_config(display="diagram")

## Preparation

First, we prepare the data, train a random forest classifier and calculate the SHAP values of this classifier for all samples in the dataset.

In [ ]:
# Set folder for data storage and loading (pre-calculated data is stored at `/original`)
folder = 'data'
os.makedirs(folder, exist_ok=True)

In [ ]:
# Load data
credit_data = pd.read_csv(f'german_credit_data.csv')
credit_data.head()

In [ ]:
# Pre-process data
data = credit_data.copy()
model_target = 'credit_risk'

numerical_features = data.drop(model_target, axis=1).select_dtypes(include='int64').columns.tolist()
categorical_features = data.drop(model_target, axis=1).select_dtypes(include=['object', 'bool']).columns.tolist()

model_features = categorical_features + numerical_features

# Convert categorical values to the integer codes so that SHAP can handle them
data[categorical_features] = data[categorical_features].astype('category').apply(lambda x: x.cat.codes)

In [ ]:
# Store pre-processed credit data
with open(f'{folder}/data.pkl', 'wb') as f:
  pickle.dump(data, f, protocol=4)

In [ ]:
# Build and train classifier
train_data, test_data = train_test_split(data, test_size=0.1, shuffle=True, random_state=23)

X_train = train_data.drop(model_target, axis=1)
y_train = train_data[model_target]

X_test = test_data.drop(model_target, axis=1)
y_test = test_data[model_target]

X_total = data.drop(model_target, axis=1)
y_total = data[model_target]

# Preprocess the numerical features
numerical_processor = Pipeline(
    [
        ("num_imputer", SimpleImputer(strategy="mean", add_indicator=True)),
        ("num_scaler", MinMaxScaler()),
    ]
)
# Preprocess the categorical features
categorical_processor = Pipeline(
    [
        ("cat_encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

# Combine all data preprocessors from above
data_processor = ColumnTransformer(
    [
        ("numerical_processing", numerical_processor, numerical_features),
        ("categorical_processing", categorical_processor, categorical_features),
    ]
)

random_forest = RandomForestClassifier(max_depth=10, random_state=1)

# Pipeline with desired data transformers, along with an estimator at the end
pipeline = Pipeline(
    [
        ("data_processing", data_processor),
        ("rf", random_forest),
    ]
)

classifier = pipeline.fit(X_train, y_train)

print(f'Accuracy (train / test): {accuracy_score(y_train, classifier.predict(X_train)):.2f} / {accuracy_score(y_test, classifier.predict(X_test)):.2f}')

# Show pipeline
pipeline

In [ ]:
# Store classifier
with open(f'{folder}/classifier.pkl', 'wb') as f:
  pickle.dump(classifier, f, protocol=4)

In [ ]:
# Calculate SHAP values
explainer = shap.Explainer(classifier.predict, X_total)
shap_values = explainer(X_total)

In [ ]:
# Show SHAP waterfall for some sample
shap.plots.waterfall(shap_values[500])

In [ ]:
# Store SHAP values
with open(f'{folder}/shap.pkl', 'wb') as f:
  pickle.dump(shap_values, f, protocol=4)

## Calculation of explanation scores

Now, we apply ICECREAM to the dataset, calculating minimum-size coalitions with (approximately) full explanation score and at most four elements for all samples.

In [ ]:
alpha = 0.9998
k = 4

coalitions = [find_minimum_size_coalitions(data, observation, 'credit_risk', classifier=classifier.predict, threshold=alpha, maximum_coalition_size=k) for _, observation in tqdm(list(data.head(50).iterrows()))]

with open(f'{folder}/coalitions.pkl', 'wb') as f:
    pickle.dump(coalitions, f, protocol=4)

## Analysis

Finally, we compare the SHAP and ICECREAM results.

In [ ]:
# Load data, classifier, SHAP values and minimum-size coalitions from /data

with open(f'{folder}/data.pkl', 'rb') as f:
  data = pickle.load(f)

model_target = 'credit_risk'

X_total = data.drop(model_target, axis=1)
y_total = data[model_target]

model_features = list(X_total.columns)


with open(f'{folder}/classifier.pkl', 'rb') as f:
    classifier = pickle.load(f)

with open(f'{folder}/shap.pkl', 'rb') as f:
    shap_values = pickle.load(f)
    shap_df = pd.DataFrame(shap_values.values, columns=model_features)
    normalized_shap_df = shap_df.div(shap_df.sum(axis=1), axis=0)

with open(f'{folder}/coalitions.pkl', 'rb') as f:
    minimum_size_coalitions = pickle.load(f)

In [ ]:
# Identify simple samples, i.e., samples which have at least one few-feature, full-explanation score coalition
simple_explanation_indices = [i for i, coalitions in enumerate(minimum_size_coalitions) if len(coalitions) > 0]

print(f'There are {len(simple_explanation_indices)}/{len(minimum_size_coalitions)} samples with simple (k <= 4, alpha >= 0.9998) explanations: {simple_explanation_indices}')

In [ ]:
# Create a list of all such coalitions, together with the sample they refer to
coalitions = pd.DataFrame([(index, coalition) for index in simple_explanation_indices for coalition in minimum_size_coalitions[index]], columns=['sample_index', 'coalition'])

In [ ]:
# Successively randomize features in three different orders and compare the prediction stability of the model

def create_randomized_samples(features, number_of_samples=1_000):
    return X_total[features].sample(n=number_of_samples, replace=True, ignore_index=True)

number_of_samples = 1_000

result_non_coalition_features, result_non_top_shap_features, result_random_features = [], [], []
for index, [sample_index, coalition] in tqdm(coalitions.iterrows(), total=len(coalitions)):
    non_top_shap_features = list(normalized_shap_df.iloc[sample_index].sort_values(ascending=True).index)
    non_coalition_features = sorted(non_top_shap_features, key=lambda feature: feature in coalition)
    random_features = model_features.copy()
    random.shuffle(random_features)

    congruence_non_coalition_features, congruence_non_top_shap_features, congruence_random_features = [], [], []
    for number_of_randomized_features in range(len(model_features) + 1):
        # Randomize in order of non_coalition_features and measure congruence
        samples = X_total.loc[[sample_index] * number_of_samples].reset_index(drop=True)
        samples[non_coalition_features[:number_of_randomized_features]] = create_randomized_samples(non_coalition_features[:number_of_randomized_features], number_of_samples=number_of_samples)

        congruence_non_coalition_features.append((classifier.predict(samples) == y_total[sample_index]).mean())

        # Randomize in order of non_top_shap_features and measure congruence
        samples = X_total.loc[[sample_index] * number_of_samples].reset_index(drop=True)
        samples[non_top_shap_features[:number_of_randomized_features]] = create_randomized_samples(non_top_shap_features[:number_of_randomized_features], number_of_samples=number_of_samples)

        congruence_non_top_shap_features.append((classifier.predict(samples) == y_total[sample_index]).mean())

        # Randomize in order of random_features and measure congruence
        samples = X_total.loc[[sample_index] * number_of_samples].reset_index(drop=True)
        samples[random_features[:number_of_randomized_features]] = create_randomized_samples(random_features[:number_of_randomized_features], number_of_samples=number_of_samples)

        congruence_random_features.append((classifier.predict(samples) == y_total[sample_index]).mean())

    result_non_top_shap_features.append(congruence_non_top_shap_features)
    result_non_coalition_features.append(congruence_non_coalition_features)
    result_random_features.append(congruence_random_features)

y_non_top_shap_features = np.array(result_non_top_shap_features)
y_non_coalition_features = np.array(result_non_coalition_features)
y_random_features = np.array(result_random_features)


In [ ]:
illustration_sample_index = 680

In [ ]:
plt.figure(figsize=(10, 4))
colors=['#009E73', '#E69F00', '#0072B2']

# Plot illustration sample
plt.plot(y_non_coalition_features[illustration_sample_index], '.-', label='Randomize non-coalition features first', color=colors[0])
plt.plot(y_non_top_shap_features[illustration_sample_index], 'x-', label='Randomize in ascending order of SHAP value', color=colors[1])
plt.plot(y_random_features[illustration_sample_index], 's-', label='Randomize in random order', color=colors[2], markersize=3)

# Plot mean and error bars for all samples
plt.plot(np.mean(y_non_coalition_features, axis=0), '--', color=colors[0])
plt.fill_between(range(len(model_features) + 1), np.quantile(y_non_coalition_features, 0.05, axis=0), np.quantile(y_non_coalition_features, 0.95, axis=0), color=colors[0], alpha=0.2)
plt.plot(np.mean(y_non_top_shap_features, axis=0), '--', color=colors[1])
plt.fill_between(range(len(model_features) + 1), np.quantile(y_non_top_shap_features, 0.05, axis=0), np.quantile(y_non_top_shap_features, 0.95, axis=0), color=colors[1], alpha=0.2)
plt.plot(np.mean(y_random_features, axis=0), '--', color=colors[2])
plt.fill_between(range(len(model_features) + 1), np.quantile(y_random_features, 0.05, axis=0), np.quantile(y_random_features, 0.95, axis=0), color=colors[2], alpha=0.2)

# Show where coalition features are randomized
plt.annotate('Start randomizing \n coalition features', xy=(18.2, 0.998), xytext=(20, 0.98), backgroundcolor='w', arrowprops=dict(arrowstyle='-|>', facecolor='gray', edgecolor='gray'))

plt.ylabel(r'Stability $\mathbb{P}[Y=y \mid do(\mathbf{V}_C = \mathbf{v}_C)]$ of prediction')
plt.xlabel('Number of randomized features')
plt.legend()

plt.savefig(f'{folder}/result.pdf', bbox_inches='tight')